In [38]:
## Program to extract interacting residues from mmCIF files within DMI and DDI curated data sets and annotate
## Created by: Joelle Strom
## Last updated: 02.02.2024

# Import libraries
import db_utils_sqlalchemy
from sqlalchemy import text
import numpy as np
import pandas as pd
from tqdm import tqdm
from Bio.PDB import PDBIO
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBParser import PDBParser
import requests
import json

# Establish connection to MySQL server
engine = db_utils_sqlalchemy.get_connection()

In [44]:
class interaction:
    """ Store information about PPI. """
    def __init__(self, pdbid=None, chain1=None, chain2=None, uniprot1=None, uniprot2=None, structure=None):
        """ Define PDB ID, chain IDs, UniProt IDs, and provide parsed structure object (expecting output of Biopython PDBParser or MMCIFParser). """
        self.pdbid = pdbid
        self.chain1 = chain1
        self.chain2 = chain2
        self.uniprot1 = uniprot1
        self.uniprot2 = uniprot2
        self.structure = structure
        self.chainids = [chain1, chain2]
    
    def get_if_res(self, cutoff=5):
       
        """ Take extracted structure from PDB file and find interface residues according to pDockQ methodology """
        
        chains = []
        for chain in self.structure.get_chains():
            if chain.get_id() in self.chainids:
                chains+=[chain]
        interface_residues1=[]
        interface_residues2=[]
        for res1 in chains[0]:
            for res2 in chains[1]:
                #Atom-atom distance
                #print (res1,res2)
                test=False
                for a in res1:
                    if test:break
                    for b in res2:
                        dist = np.linalg.norm(a.coord - b.coord)
                        if dist < cutoff:
                            #Save residues
                            #print ("Appending",res1,res2)
                            interface_residues1.append(res1.id[1])
                            interface_residues2.append(res2.id[1])
                            test=True
                            break
                        elif dist > 2*cutoff: # To speed up things
                            test=True
                            break
        if [chain.get_id() for chain in chains] == self.chainids:
            ifres_motif = np.unique(interface_residues1)
            ifres_domain = np.unique(interface_residues2)
        else:
            ifres_motif = np.unique(interface_residues2)
            ifres_domain = np.unique(interface_residues1)    
        return ifres_motif, ifres_domain
    
    def get_newres(self, chainnum, mapdf, ifres):  
       
        """ Shift interface residue numbers according to information from Uniprot-PDB mapping. """
        
        pdb_id = self.pdbid
        if chainnum == 1:
            chain_id = self.chain1
        elif chainnum == 2:
            chain_id = self.chain2
        else:
            return("Desired chain outside of range.")
        chainmap = mapdf.query('pdb_id == @pdb_id and chain_id == @chain_id')
        newres = []
        for j,s in chainmap.iterrows():
            temp = [(x + s['shift']) for x in ifres if x in range(s['pdb_start'], s['pdb_end']+1)]
            newres+=temp
        return newres
    
    def get_dis_pred(self, chainnum, prot):
    
        """ Use row index and obtained interface residue arrays to obtain disorder prediction. """

        if chainnum == 1:
            uniprot = self.uniprot1
        elif chainnum == 2:
            uniprot = self.uniprot2
        else:
            return("Desired chain outside of range.")
        if len(prot) > 0 and uniprot != 'No UniProt':
            # Access IUPred2A REST API
            url = 'http://iupred2a.elte.hu/iupred2a/short/'+uniprot+'.json'
            response = requests.get(url)
            res = response.json()
            iupred = pd.Series(res['iupred2'])
            if len(iupred.index) > 0:
                iupred.index = iupred.index + 1
                if len(list(set(prot).difference(iupred.index))) == 0:
                    # Restrict disorder predictions to only the residues which are in interface
                    iupredres = iupred[prot]
                    numdis = len(iupredres[iupredres > 0.4])
                    numres = len(prot)
                else:
                    print("Desired residue index is outside range of IUPred predictions based on this UniProt ID:",uniprot)
                    numdis = np.nan
                    numres = len(prot)
            else:
                numdis = np.nan
                numres = len(prot)
        else:
            numdis = np.nan
            numres = len(prot)
            iupred = pd.Series([])
        return numdis, numres, iupred

In [3]:
# Import curated DMI data set
query = text('SELECT * FROM chopyan_db.AlphaFold_minimal_PRS_DMI_structure_info WHERE for_AF2_benchmark = 1')
dmi_df = pd.read_sql(query, con=engine)
# Manual replacement of UniProt IDs for one interaction based on investigation after IUPred accession issues
dmi_df.loc[dmi_df['pdb_id'] == '1O6K','uniprot_motif'] = 'P49841'
dmi_df.loc[dmi_df['pdb_id'] == '1O6K','uniprot_domain'] = 'P31751'

# Import curated DDI dataset
ddi_df = pd.read_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_dataset.csv')
ddi_df.reset_index(drop=True,inplace=True)

# Uniprot-PDB mapping dataset (from Uniprot_PDB_Mapping.ipynb)
unp_pdb_mapp = pd.read_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/UniProt_PDB_Mapping.csv')

In [4]:
ddi_df

,Unnamed: 0,DDI_type,PDB_ID,Chain_ID1,DomainID1,DomainName1,DomainStart1,DomainEnd1,Chain_ID2,DomainID2,...,chain1_cryst_frac_iface_above04,chain2_cryst_frac_iface_above04,chain1_uniprot_avg_iface_iupred,chain2_uniprot_avg_iface_iupred,chain1_uniprot_frac_iface_above04,chain2_uniprot_frac_iface_above04,chain1_num_iface_res,chain2_num_iface_res,chain1_unmapped_iface_res,chain2_unmapped_iface_res
0,0,PF00004_PF10584,6EPF,I,PF00004,AAA,222,355,C,PF10584,...,0.400000,0.166667,0.235960,0.189883,0.000000,0.000000,5,6,0.0,0.0
1,1,PF00009_PF01873,2D74,A,PF00009,GTP_EFTU,9,201,B,PF01873,...,0.000000,0.133333,0.170565,0.109300,0.000000,0.000000,17,15,0.0,0.0
2,2,PF00010_PF02344,6G6L,B,PF00010,HLH,214,254,C,PF02344,...,1.000000,0.571429,0.515222,0.102229,1.000000,0.000000,9,7,0.0,0.0
3,3,PF00019_PF00041,4UI2,B,PF00019,TGF_beta,294,395,A,PF00041,...,0.000000,0.100000,0.064760,0.292040,0.000000,0.000000,10,10,0.0,0.0
4,4,PF00023_PF07686,4NIK,A,PF00023,Ank,105,137,B,PF07686,...,0.666667,0.000000,0.377500,NaN,0.333333,NaN,3,7,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,75,PF16004_PF00400,6ID0,C,PF16004,EFTUD2,56,110,T,PF00400,...,0.900000,0.250000,0.501250,0.108217,0.900000,0.000000,10,12,0.0,0.0
76,76,PF17292_PF08644,4KHB,H,PF17292,POB3_N,5,97,G,PF08644,...,0.000000,0.000000,0.173667,0.233467,0.000000,0.000000,6,3,0.0,0.0
77,77,PF17820_PF16523,3L4F,D,PF17820,PDZ_6,701,755,A,PF16523,...,0.250000,1.000000,0.270425,0.791320,0.000000,1.000000,4,5,0.0,0.0
78,78,PF17838_PF00071,3KZ1,B,PF17838,PH_16,950,1079,E,PF00071,...,0.000000,0.166667,0.053487,0.179067,0.000000,0.166667,15,12,0.0,0.0


In [15]:
dmi_df

,dmi_type,regex,binary,modified_residue,pdb_id,methods,organism,uniprot_motif,uniprot_domain,chain_motif,...,uniprot_motif_start,uniprot_motif_end,checked_by,for_AF2_benchmark,comments,comments_after_prediction,Is_intramolecular,unsolved_residue_in_motif,motif_secondary_structure,regex_for_defined_position_function
0,DEG_APCC_KENBOX_2,.KEN.,1.0,0,4GGD,mutation analysis; pull down; x-ray crystallog...,Homo sapiens,O60566,Q12834,D,...,25,29,"DM, KL",1,"This structure is crystollagraphic (x-ray), it...",,None,NaN,L,
1,DEG_COP1_1,"[STDE]{1,3}.{0,2}[TSDE].{2,3}VP[STDE]G{0,1}[FL...",1.0,0,5IGO,coimmunoprecipitation; competition binding; fl...,Homo sapiens,Q96RU8,P43254,X,...,354,361,JL,1,"In the uniprot sequence, the HMMs of WD40 matc...",,None,NaN,L,
2,DEG_Kelch_Keap1_1,[DNS].[DES][TNS]GE,1.0,0,2FLU,alanine scanning; coimmunoprecipitation; compe...,Homo sapiens,Q16236,Q14145,P,...,77,82,"KL, CS",1,,,None,NaN,L,
3,DEG_Kelch_Keap1_2,QD.DLGV,1.0,0,3WN7,alanine scanning; glutathione s tranferase tag...,Mus musculus,Q60795,Q9Z2X8,B,...,26,32,CS,1,,,None,NaN,H,
4,DEG_MDM2_SWIB_1,"F[^P]{3}W[^P]{2,3}[VIL]",1.0,0,1YCR,fluorescence polarization spectroscopy; isothe...,Homo sapiens,P04637,Q00987,B,...,19,26,JL,1,,,None,NaN,H,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,LIG_ActinCP_TwfCPI_2,"F.[KR]P..[PAS].{0,3}[RK]",NaN,0,7DS2,colocalization; comigration in non denaturing ...,Mus musculus,Q91YR1,P14315,C,...,323,333,dm,1,no modified residues; It seems that motif bind...,,None,NaN,L,
132,LIG_KLC1_Yacidic_2,"[ED].{0,1}[IYVLMTF]Y[LIV][DE]",NaN,0,6FUZ,alanine scanning; coimmunoprecipitation; fluor...,Homo sapiens,Q9UQF2,O88447,A,...,707,711,dm,1,"it is a new DMI type, I checked ELM and typed ...",dm:predicted model showed that the motif is in...,None,NaN,L,
133,LIG_LYPXL_SIV_4,[PA]Y..[AV][^P]{3}L,NaN,0,2XS1,biosensor; mutation analysis; western blot; x-...,Simian immunodeficiency virus - mac K6W,P05893,Q8WUM4,B,...,487,495,dm,1,"a new DMI type, I put regex from ELM; no bound...",,None,NaN,H,
134,TRG_DiLeu_BaEn_1,E..[^P]L[LIVM],NaN,0,4NEE,isothermal titration calorimetry; mutation ana...,HIV-1 M:B_HXB2R,P04601,P62744,C,...,160,165,"dm, JL",1,"a new DMI type, i filled the regex from ELM; n...","dm run19: prediction did not look good, the pr...",None,NaN,L,


In [16]:
## OBTAIN STRUCTURE FILES FROM PDB
# Downloaded on 27.10.2023
#for i,r in tqdm(ddi_df.iterrows()):
    #pdb_id_toget = r['PDB_ID'].lower()
    #url = f'https://files.rcsb.org/view/{pdb_id_toget}.cif'
    #f = open(f'/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_structures/{pdb_id_toget}.cif', 'w')
    #f.write(requests.get(url).text)
    #f.close()

# Downloaded on 31.10.2023
#for i,r in tqdm(dmi_df.iterrows()):
    #pdb_id_toget = r['pdb_id'].lower()
    #url = f'https://files.rcsb.org/view/{pdb_id_toget}.cif'
    #f = open(f'/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_structures/{pdb_id_toget}.cif', 'w')
    #f.write(requests.get(url).text)
    #f.close()

In [45]:
#from myMMCIFParser import MMCIFParser
bio_parser = MMCIFParser(auth_residues=False, QUIET=True)
if_numdis1 = []
if_numdis2 = []
if_numres1 = []
if_numres2 = []
dmi_dict = {}

# Collect interface residues and corresponding disorder predictions for all interactions in DMI dataframe
for i,r in tqdm(dmi_df.iterrows()):
    name = r['pdb_id'].lower()
    filename = f'/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_structures/{name}.cif'
    structure = bio_parser.get_structure(name, filename)
    intobj = interaction(pdbid=r['pdb_id'], chain1=r['chain_motif'], chain2=r['chain_domain'], 
                         uniprot1=r['uniprot_motif'], uniprot2=r['uniprot_domain'], structure=structure)
    if intobj.chain1 != intobj.chain2:
        ifres1, ifres2 = intobj.get_if_res(10)
        newres1 = intobj.get_newres(chainnum=1, mapdf=unp_pdb_mapp, ifres=ifres1)
        newres2 = intobj.get_newres(chainnum=2, mapdf=unp_pdb_mapp, ifres=ifres2)
        numdis1, numres1, iupred1 = intobj.get_dis_pred(chainnum=1, prot=newres1)
        numdis2, numres2, iupred2 = intobj.get_dis_pred(chainnum=2, prot=newres2)
        if_numdis1.append(numdis1)
        if_numdis2.append(numdis2)
        if_numres1.append(numres1)
        if_numres2.append(numres2)
        dmi_dict[intobj.pdbid] = {'chain1':intobj.chain1,
                                  'chain2':intobj.chain2,
                                  'uniprot1':intobj.uniprot1,
                                  'uniprot2':intobj.uniprot2,
                                  'ifres1':ifres1.tolist(),
                                  'ifres2':ifres2.tolist(),
                                  'newres1':newres1,
                                  'newres2':newres2,
                                  'iupred1':iupred1.tolist(),
                                  'iupred2':iupred2.tolist()}
    else:
        if_numdis1.append(np.nan)
        if_numdis2.append(np.nan)
        if_numres1.append(np.nan)
        if_numres2.append(np.nan)
        dmi_dict[intobj.pdbid] = {'chain1':intobj.chain1,
                                  'chain2':intobj.chain2,
                                  'uniprot1':intobj.uniprot1,
                                  'uniprot2':intobj.uniprot2,
                                  'ifres1':[],
                                  'ifres2':[],
                                  'newres1':[],
                                  'newres2':[],
                                  'iupred1':[],
                                  'iupred2':[]}

# Append number of residues in interface from each chain and number of IF residues with IUPred2A disorder prediction to dataframe
dmi_df['if_numdis1'] = if_numdis1
dmi_df['if_numdis2'] = if_numdis2
dmi_df['if_numres1'] = if_numres1
dmi_df['if_numres2'] = if_numres2

136it [06:00,  2.65s/it]


In [ ]:
# 1ATP,1MKE,6FUZ are intramolecular, meaning chain_motif and chain_domain are the same. These DMIs are skipped for now
# P49841 was not initially in index for IUPred2A predictions (this issue has since been fixed)

In [50]:
# Write to disk
dmi_df.to_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_dataset_with_IUPred.csv', index=False)
f = open('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_IF_annotations.json', 'w')
f.close()

def convert(o):
    if isinstance(o, np.int32): return int(o)
    elif isinstance(o, np.int64): return int(o)
    elif isinstance(o, np.nan): return 'NaN'
    raise TypeError

with open('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_IF_annotations.json', 'w') as outfile: 
    outfile.write(json.dumps(dmi_dict, default=convert))

In [51]:
if_numdis1 = []
if_numdis2 = []
if_numres1 = []
if_numres2 = []
ddi_dict = {}

# Collect interface residues and corresponding disorder predictions for all interactions in DDI dataframe
for i,r in tqdm(ddi_df.iterrows()):
    name = r['PDB_ID'].lower()
    filename = f'/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_structures/{name}.cif'
    structure = bio_parser.get_structure(name, filename)
    intobj = interaction(pdbid=r['PDB_ID'], chain1=r['Chain1'], chain2=r['Chain2'], 
                         uniprot1=r['UniProt1'], uniprot2=r['UniProt2'], structure=structure)
    if intobj.chain1 != intobj.chain2:
        ifres1, ifres2 = intobj.get_if_res(10)
        newres1 = intobj.get_newres(chainnum=1, mapdf=unp_pdb_mapp, ifres=ifres1)
        newres2 = intobj.get_newres(chainnum=2, mapdf=unp_pdb_mapp, ifres=ifres2)
        numdis1, numres1, iupred1 = intobj.get_dis_pred(chainnum=1, prot=newres1)
        numdis2, numres2, iupred2 = intobj.get_dis_pred(chainnum=2, prot=newres2)
        if_numdis1.append(numdis1)
        if_numdis2.append(numdis2)
        if_numres1.append(numres1)
        if_numres2.append(numres2)
        ddi_dict[intobj.pdbid] = {'chain1':intobj.chain1,
                                  'chain2':intobj.chain2,
                                  'uniprot1':intobj.uniprot1,
                                  'uniprot2':intobj.uniprot2,
                                  'ifres1':ifres1.tolist(),
                                  'ifres2':ifres2.tolist(),
                                  'newres1':newres1,
                                  'newres2':newres2,
                                  'iupred1':iupred1.tolist(),
                                  'iupred2':iupred2.tolist()}
    else:
        if_numdis1.append(np.nan)
        if_numdis2.append(np.nan)
        if_numres1.append(np.nan)
        if_numres2.append(np.nan)
        ddi_dict[intobj.pdbid] = {'chain1':intobj.chain1,
                                  'chain2':intobj.chain2,
                                  'uniprot1':intobj.uniprot1,
                                  'uniprot2':intobj.uniprot2,
                                  'ifres1':[],
                                  'ifres2':[],
                                  'newres1':[],
                                  'newres2':[],
                                  'iupred1':[],
                                  'iupred2':[]}

# Append number of residues in interface from each chain and number of IF residues with IUPred2A disorder prediction to dataframe
ddi_df['if_numdis1'] = if_numdis1
ddi_df['if_numdis2'] = if_numdis2
ddi_df['if_numres1'] = if_numres1
ddi_df['if_numres2'] = if_numres2

80it [04:42,  3.53s/it]


In [52]:
# Write to disk
ddi_df.to_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_dataset_with_IUPred.csv', index=False)

with open('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_IF_annotations.json', 'w') as outfile: 
    outfile.write(json.dumps(ddi_dict, default=convert))

In [33]:
# Check some data issues

df = pd.read_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DMI_dataset_with_IUPred.csv')
df.query('if_numres1 == 0 or if_numres2 == 0')

df = pd.read_csv('/mnt/c/Users/stromjoe/Documents/projects/DDI_IF-Analysis/DDI_dataset_with_IUPred.csv')
df.query('if_numres1 == 0 or if_numres2 == 0')

,dmi_type,regex,binary,modified_residue,pdb_id,methods,organism,uniprot_motif,uniprot_domain,chain_motif,...,comments,comments_after_prediction,Is_intramolecular,unsolved_residue_in_motif,motif_secondary_structure,regex_for_defined_position_function,if_numdis1,if_numdis2,if_numres1,if_numres2
13,DOC_MAPK_DCC_7,"([RK].{2,4}[LIVP]P.[LIV].[LIVMF])|([RK].{2,4}[...",1.0,0,2B9J,fluorescence polarization spectroscopy; glutat...,Saccharomyces cerevisiae S288c,P21268,P16892,C,...,one ADP in structure. JL: The domain and motif...,NaN,NaN,NaN,L,"[RK].{2,4}[LIVP].P[LIV].[LIVMF]",NaN,12.0,0.0,53.0
41,LIG_CAP-Gly_1,"[ED].{0,2}[ED].{0,2}[EDQ].{0,1}[YF]$",1.0,0,2PZO,coimmunoprecipitation; glutathione s tranferas...,Homo sapiens,P30622,Q14203,E,...,several chains are present in the pdb structur...,NaN,NaN,NaN,L,NaN,NaN,NaN,0.0,0.0
44,LIG_Clathr_ClatBox_1,L[IVLMF].[IVLMF][DE],1.0,0,1C9I,x-ray crystallography,Homo sapiens,O00203,P11442,C,...,several chains are present in the pdb structur...,dm run19: AF2 predicted the domain perfectly c...,NaN,NaN,L,NaN,NaN,8.0,0.0,48.0
48,LIG_CSL_BTD_1,[AFILMPTVW]W[FHILMPSTVW]P,1.0,0,4J2X,isothermal titration calorimetry; mutation ana...,Mus musculus,A2AEX7,P31266,D,...,"one bound ligand EDO, several chains are prese...","After AF2 run, the predicted model looked not ...",NaN,NaN,L,NaN,NaN,0.0,0.0,71.0
52,LIG_EH_1,.NPF.,1.0,0,1FF1,classical fluorescence spectroscopy; coimmunop...,Homo sapiens,P52594,P42566,B,...,bound Ca ions are present in the structure,NaN,NaN,NaN,L,NaN,NaN,0.0,0.0,48.0
53,LIG_EH1_1,.[FYH].[IVM][^WFYP][^WFYP][ILM][ILMV].,1.0,0,2CE8,x-ray crystallography,Homo sapiens,P56915,Q04724,X,...,crystal structure is a dimer that has a dimeri...,dm run14: after AF2 run we could see that dom...,NaN,NaN,H,NaN,NaN,0.0,0.0,74.0
54,LIG_EVH1_1,([FYWL]P.PP)|([FYWL]PP[ALIVTFY]P),1.0,0,1EVH,classical fluorescence spectroscopy; mutation ...,Listeria monocytogenes,P33379,Q03173,B,...,no bound ligands and ions and no modified resi...,AF2 run (#14) ouctome: the domain as well as t...,NaN,NaN,L,NaN,NaN,0.0,0.0,37.0
58,LIG_FXI_DFP_1,[FYWHIL].DF[PD],1.0,0,5EOD,surface plasmon resonance; x-ray crystallography,Homo sapiens,P25391,P03951,B,...,"one bound ligand -NAG, and no modified residue...",AF2 run (#14) showed that motif was pushed awa...,NaN,NaN,L,NaN,NaN,0.0,0.0,49.0
59,LIG_GBD_Chelix_1,[ILV][VA][^P][^P][LI][^P][^P][^P][LM],1.0,0,2K42,alanine scanning; deletion analysis; isotherma...,Escherichia coli O157:H7 str. TW14359,C6UYI3,P42768,B,...,This structure shows intermolecular interactio...,NaN,NaN,NaN,H,NaN,NaN,25.0,0.0,58.0
77,LIG_NRP_CendR_1,"[RK].{0,2}R$",1.0,0,2ORZ,x-ray crystallography,Homo sapiens,P01858,Q9QWJ9,B,...,no bound ligands and no modified residues; sin...,dm: the domain is superimposed perfectly with ...,NaN,1.0,L,NaN,NaN,3.0,0.0,38.0
